In [2]:
import pandas as pd
from itertools import product
import random
import numpy as np

## Metadata dataframe

In [15]:
table_set_loaded = '2017 Loaded CSO Preferred Structure'
table_set_unloaded = '2017 Unloaded CSO Preferred Structure'

tuples = [
    (3299, table_set_loaded, "NS_SP", "Male"),
    (3300, table_set_loaded, "NS_P", "Male"),
    (3301, table_set_loaded, "NS_R", "Male"),
    (3302, table_set_loaded, "NS_SP", "Female"),
    (3303, table_set_loaded, "NS_P", "Female"),
    (3304, table_set_loaded, "NS_R", "Female"),
    (3305, table_set_loaded, "S_P", "Male"),
    (3306, table_set_loaded, "S_R", "Male"),
    (3307, table_set_loaded, "S_P", "Female"),
    (3308, table_set_loaded, "S_R", "Female"),
]

meta_df = pd.DataFrame(tuples, columns=['TableNumber', 'TableSet', 'Underwriting', 'Sex'])
meta_df.reset_index(inplace=True)
meta_df.to_csv('assumption_tables/2017_loaded_CSO_index.csv', index=False)
meta_df

,index,TableNumber,TableSet,Underwriting,Sex
0,0,3299,2017 Loaded CSO Preferred Structure,NS_SP,Male
1,1,3300,2017 Loaded CSO Preferred Structure,NS_P,Male
2,2,3301,2017 Loaded CSO Preferred Structure,NS_R,Male
3,3,3302,2017 Loaded CSO Preferred Structure,NS_SP,Female
4,4,3303,2017 Loaded CSO Preferred Structure,NS_P,Female
5,5,3304,2017 Loaded CSO Preferred Structure,NS_R,Female
6,6,3305,2017 Loaded CSO Preferred Structure,S_P,Male
7,7,3306,2017 Loaded CSO Preferred Structure,S_R,Male
8,8,3307,2017 Loaded CSO Preferred Structure,S_P,Female
9,9,3308,2017 Loaded CSO Preferred Structure,S_R,Female


## Generate a tidy dataframe

Attempt to make a format that is similar to the format from the HeavyTable library

In [4]:
from pymort import MortXML

issue_attained_tuples = [(x, y, y-x+1) for x, y in product(range(18, 121), repeat=2) if y >= x]
template_df = pd.DataFrame(issue_attained_tuples, columns=['IssueAge', 'Age', 'Duration'])
tidy_format = pd.merge(template_df.assign(key=0), meta_df.assign(key=0), on='key').sort_values(['TableNumber', 'Age', 'Duration'])
select_rate_dfs = []
ultimate_rate_dfs = []
for table_number in meta_df['TableNumber']:
    select_rate_df = MortXML.from_id(table_number).Tables[0].Values
    select_rate_df.reset_index(inplace=True)
    ultimate_rate_df = MortXML.from_id(table_number).Tables[1].Values
    ultimate_rate_df.reset_index(inplace=True)
    select_rate_df.rename(columns={"vals": "select_vals", "Age": "IssueAge"}, inplace=True)
    ultimate_rate_df.rename(columns={"vals": "ultimate_vals"}, inplace=True)
    select_rate_df.reset_index(inplace=True)
    ultimate_rate_df.reset_index(inplace=True)
    select_rate_df["TableNumber"] = table_number
    ultimate_rate_df["TableNumber"] = table_number
    select_rate_dfs.append(select_rate_df)
    ultimate_rate_dfs.append(ultimate_rate_df)

select_rate_df = pd.concat(select_rate_dfs)
print(select_rate_df.columns)
ultimate_rate_df = pd.concat(ultimate_rate_dfs)

final_df = tidy_format \
    .merge(select_rate_df, on=['TableNumber', 'IssueAge', 'Duration'], how='left') \
    .merge(ultimate_rate_df, on=['TableNumber', 'Age'], how='left')

# use ultimate_vals if vals is null
final_df['vals'] = final_df['select_vals'].fillna(final_df['ultimate_vals'])
final_df = final_df[['IssueAge','Age', 'Duration', 'Underwriting', 'Sex', 'vals']].sort_values(['Underwriting', 'Sex', 'IssueAge', 'Age', 'Duration'])
final_df.to_csv('assumption_tables/2017_loaded_CSO_mortality_rates.csv', index=False)

Index(['index', 'IssueAge', 'Duration', 'select_vals', 'TableNumber'], dtype='object')


## Generate an array

Store directly as NumPy, assume users will linear index into the mortality table with no string lookups

In [7]:
select = np.array([MortXML.from_id(id).Tables[0].Values.unstack().values for id in meta_df['TableNumber']])
ultimate = np.array([MortXML.from_id(id).Tables[1].Values.unstack().values for id in meta_df['TableNumber']])
with open('assumption_tables/2017_loaded_CSO_mortality_rates_select.npy', 'wb') as f:
    np.save(f, select)
with open('assumption_tables/2017_loaded_CSO_mortality_rates_ultimate.npy', 'wb') as f:
    np.save(f, ultimate)


In [11]:
pd.DataFrame(select[0])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.00070,0.00072,0.00075,0.00078,0.00075,0.00072,0.00072,0.00072,0.00061,0.00058,...,0.00057,0.00065,0.00075,0.00088,0.00099,0.00109,0.00115,0.00120,0.00127,0.00135
1,0.00065,0.00068,0.00069,0.00065,0.00065,0.00066,0.00066,0.00061,0.00053,0.00046,...,0.00058,0.00067,0.00078,0.00090,0.00102,0.00112,0.00118,0.00125,0.00133,0.00141
2,0.00056,0.00057,0.00063,0.00061,0.00059,0.00058,0.00056,0.00051,0.00046,0.00046,...,0.00060,0.00070,0.00080,0.00092,0.00105,0.00115,0.00121,0.00127,0.00135,0.00145
3,0.00047,0.00057,0.00051,0.00049,0.00048,0.00047,0.00046,0.00043,0.00040,0.00043,...,0.00063,0.00072,0.00082,0.00095,0.00107,0.00115,0.00121,0.00128,0.00138,0.00149
4,0.00045,0.00047,0.00043,0.00040,0.00039,0.00039,0.00038,0.00037,0.00036,0.00041,...,0.00065,0.00074,0.00085,0.00098,0.00110,0.00116,0.00122,0.00128,0.00140,0.00154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.02368,0.06223,0.14189,0.22673,0.24340,0.26243,0.28269,0.30471,0.32800,0.35209,...,0.47743,0.50332,0.53061,0.55939,0.58972,0.62170,0.65542,0.69096,0.72843,0.76794
74,0.03362,0.09526,0.22673,0.24340,0.26243,0.28269,0.30471,0.32800,0.35209,0.37447,...,0.50332,0.53061,0.55939,0.58972,0.62170,0.65542,0.69096,0.72843,0.76794,0.80958
75,0.04979,0.14355,0.24340,0.26243,0.28269,0.30471,0.32800,0.35209,0.37447,0.39680,...,0.53061,0.55939,0.58972,0.62170,0.65542,0.69096,0.72843,0.76794,0.80958,0.85348
76,0.07304,0.16782,0.26243,0.28269,0.30471,0.32800,0.35209,0.37447,0.39680,0.41866,...,0.55939,0.58972,0.62170,0.65542,0.69096,0.72843,0.76794,0.80958,0.85348,0.89977


## Generate modelpoints

In [6]:
random.seed(0)
selection = random.choices(list(product(set(meta_df["Underwriting"]), set(meta_df["Sex"]), range(18, 50), range(1, 20))), k=100_000)
selection
mp_df = pd.DataFrame(selection, columns=['Underwriting', 'Sex', 'IssueAge', 'Duration'])

mp_df.to_csv('modelpoints_tables/2017_loaded_CSO_modelpoints.csv', index=False)